In [1]:
import streamlit as st
import os
import os.path

from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage, ServiceContext, StorageContext
from llama_index.llms.openai import OpenAI

In [2]:
load_dotenv()
storage_path = "./vectorstore"
documents_path = "./documents"

llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

C:\Users\darsh\AppData\Local\Temp\ipykernel_11436\512425834.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm)


In [10]:
@st.cache_resource(show_spinner=False)
def initialize(): 
    if not os.path.exists(storage_path):
        documents = SimpleDirectoryReader(documents_path).load_data()
        index = VectorStoreIndex.from_documents(documents)
        index.storage_context.persist(persist_dir=storage_path)
    else:
        storage_context = StorageContext.from_defaults(persist_dir=storage_path)
        index = load_index_from_storage(storage_context)
    return index
index = initialize()

In [13]:
# Retrieve input (e.g., from user input)
input_text = "what should be the daily protien intake for a 60 kg man"

In [14]:
query_engine = index.as_query_engine()
response = query_engine.query(
    input_text
)

The daily protein intake for a 60 kg man should be 60 grams.


In [19]:
print(response)

The daily protein intake for a 60 kg man should be 60 grams.


In [20]:
query_engine = index.as_query_engine()
# Query the index
results = query_engine.query(input_text)

# Process results (e.g., select the most relevant result)
if results:
    response = results
else:
    response = "Sorry, I couldn't find a relevant response."

# Display response
print("Response:", response)

Response: The daily protein intake for a 60 kg man should be 60 grams.


# Streamlit part

In [21]:
st.title("Ask the Document")
if "messages" not in st.session_state.keys():
    st.session_state.messages = [
        {"role": "assistant", "content": "Ask me a question !"}
    ]

In [22]:
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)

if prompt := st.chat_input("Your question"): 
    st.session_state.messages.append({"role": "user", "content": prompt})

for message in st.session_state.messages: 
    with st.chat_message(message["role"]):
        st.write(message["content"])

AttributeError: st.session_state has no attribute "messages". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization

In [4]:
if st.session_state.messages[-1]["role"] != "assistant":
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = chat_engine.chat(prompt)
            st.write(response.response)
            pprint_response(response, show_source=True)
            message = {"role": "assistant", "content": response.response}
            st.session_state.messages.append(message) 

AttributeError: st.session_state has no attribute "messages". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization